### South Atlantic Amphibian and reptile areas

This indicator was used in the 2023 base blueprint.

In 2022, it was not used in all zonation areas where the source data overlaps, so we added a step to clip it down to the area where it was used in zonation. 

In 2023, we fixed the extent issue and we also made the marine area NoData instead of zero, to better represent where these data are relevant. This code makes an output for the full extent.

This indicator was originally made for the South Atlantic Blueprint.

Created by Amy Keister, last run by Amy Keister on 10 May 2023. It took 10 minutes to run

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SA_AmphibRept\SouthAtlanticAmphi.gdb"

In [4]:
# define final indicator outputs
out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SA_AmphibRept\SouthAtlanticAmphibianAndReptileAreas2023.tif"

In [5]:
# define sub-indicator outputs

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [7]:
# define inputs
parca= r"F:\GIS_DATA\SpeciesAndHabitats\PARCA\PARCA.gdb\PARCAs_p_dissolve"
SAraster= r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SA2021extent.tif"
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"

### Start analysis

In [8]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [9]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SA_AmphibRept\SouthAtlanticAmphi.gdb


### Bring in polygon PARCA data

In [10]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SAraster, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster(parca, "raster", "parcaR", cell_size=Rextent)

In [11]:
# reclassify to make NoData areas zero
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SAraster, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.Reclassify("parcaR", "VALUE", "1 2;NODATA 0", "DATA"); out_raster.save("parcaR0")

### Add zeros for study area and remove marine

In [12]:
# reclassify SA raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SAraster, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify(SAraster, "VALUE", "1 0;NODATA NODATA", "DATA"); out_raster.save("sa")

In [13]:
# use South Atlantic extent to start the processof adding zeros
#use cell statistics max to combine above rasters
out_raster = arcpy.sa.CellStatistics("SA;parcaR", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save(r"parcaRsa")

In [14]:
# to help with display in the Blueprint explorer, we are trying to have more consistant rules for when things are zero
# and when they are nodata. Because the PARCA data only considered terrestrial species, it would make more sense to 
# have the marine areas be nodata. Since this project didn't designate a study area, we are using the old 
#We are using NLCD to remove the deep marine areas, just because it is quick and easy and we can 
# consistantly do it for several indicators like this.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(nlcd, "parcaRsa", None, "Value > 0"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

for this one, since it isn't binned, we can't calculate a descript field.

In [15]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 1:
        return '1 = Priority Amphibian and Reptile Conservation Area (PARCA)'
    elif v == 0:
        return '0 = Not a Priority Amphibian and Reptile Conservation Area (PARCA)'
"""

In [16]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SA_AmphibRept\\SouthAtlanticAmphi.gdb\\indicator'>

In [17]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 1:
		return 56
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 1:
		return 168
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 1:
		return 0
	if Value == 0:
		return 255
	else:
		return 255
"""

In [18]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SA_AmphibRept\\SouthAtlanticAmphi.gdb\\indicator'>

In [19]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask")

In [21]:
# export as .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask", out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [22]:
end = time.time()
print(end - start)

569.5069105625153
